## Microgrid Formation

In the modern grid, the line between consumer and producer is blurred due to consumers that have distributed generation and storage resources, so to graph an electrical grid we will call the nodes **prosumers**. Some areas with a significant amount of local distributed resources are able to be self-sufficient, and it can be in their interest to form a **microgrid** that is managed independently from the central grid.

We want to identify $K$ self-sufficient microgrid communities indexed by $l$. In order to do so, we must both maximize *modularity*, the degree to which microgrids are more connected to themselves than to other microgrids, and *self-reliance*, how well supply matches demand within all microgrids. In addition to these, we must find an optimal number of communities $K$, and so, we can turn this optimization problem into a QUBO model with $K\cdot|V|$ binary bariables $x_{i,l}$ that signifies node $i$ belonging to community $l$. 

The following derivation is found in [Community Detection in Electrical Grids Using Quantum Annealing (2021)](https://arxiv.org/abs/2112.08300) and formulated in section 3 of [Quantum Optimization for the Future Energy Grid: Summary and Quantum Utility Prospects (2024)](https://arxiv.org/abs/2403.17495).


We want to maximize the modularity of a network, defined as

$$\mathcal{M} = \frac1{2m} \sum_{i,j} \left(A_{ij}-\frac{k_ik_j}{2m}\right) \sum_l x_{i,l}x_{j,l}$$
where $A_{ij}$ is adjacency matrix, $k_i=\sum_j A_{ij}$ is the sum of all edge weights from node $i$, $m=\frac12 \sum_i k_i=\frac12 \sum_i\sum_j A_{ij}=\sum_{i\neq j} A_{ij}$ is the sum of all edge weights.

The weights $A_{ij}$ are computed based on load and production to form microgrids. Each node has a "power consumption value" $p_i$ signifying how much power it produces to (+) or consumes from (-) the grid.

To optimize self-sufficiency, we want the power produced to be equal to the power consumed within each microgrid, equivalent to minimizing 
$$\mathcal{S} = \frac{1}{\mathcal{P}}\sum_l\left(\sum_i p_i x_{i,l}\right)^2$$

where $\mathcal{P}$ is a normalization constant, let's say $\max_i (p_i)^2$. 

In full, the optimization problem involves minimizing the following objective function over $KN$ variables $x_{i,l}$
$$f =\mathcal{M}+\lambda \mathcal{S} = \frac1{2m} \sum_{i,j} \left(A_{ij}-\frac{k_ik_j}{2m}\right) \sum_l x_{i,l}x_{j,l} + \frac{1}{\mathcal{P}}\sum_l\left(\sum_i p_i x_{i,l}\right)^2 $$

where $\lambda$ is tuned to balance optimization between both modularity and self-sufficiency, to find $K$ self-sufficient microgrids. 

We can classically apply the [Leiden algorithm](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6435756/) to partition the network into $K$ microgrids, an NP-hard algorithm that scales exponentially with the size of the graph. Solved by dynamic programming in $\mathcal{O}(n^3)$ time for $n$ nodes. 
When formulating this problem on a quantum computer using QUBO, we can optimize on all nodes directly, and reduce the algorithm to run in polynomial time with the size of the graph, as we need to encode only the polynomial many edges. 
Instead of formulating $KN$ binary variables $x_{i,l}$ and running the algorithm by incrementing $K$ by one from $K=2$, we can run the same algorithm with binary-greedy method. 

To do this, we continuously partition the grid into $K=2$ groups. This allows us to use just a single binary variable for each node $x_{i}$ to represent one of two groups, repeating the algorithm $\log_2(\max(K))$ times with $N$ nodes. Thus at each step we re-formulate the problem parts as follows:

$$\begin{aligned}
\mathcal{M} &= \frac1{2m} \sum_{i,j} \left(A_{ij}-\frac{k_ik_j}{2m}\right) \left(x_{i}x_{j}+(1-x_i)(1-x_j)\right) 
\\ &= \frac1{m} \sum_{i\neq j} \left(A_{ij}-\frac{k_ik_j}{2m}\right) \left(2x_{i}x_{j}-x_i-x_j+1\right)
\\ \mathcal{S} &= \frac{1}{\mathcal{P}}\left[\left(\sum_i p_i x_{i}\right)^2 + \left(\sum_i p_i (1-x_{i})\right)^2\right] 
\end{aligned}
$$
